In [ ]:
!pip install pyscipopt pyscipopt-ml -q

# Пример задачи

(этот пример основан на примере от авторов фреймворка)

Производитель автомобилей хочет определить технические параметры новой машины так, чтобы максимизировать доход от продаж. При этом производитель хочет, чтобы эта машина была малогабаритная.  

У нас есть датасэт с объемами продаж и стоимостями других машин и их технические параметрами. Мы обучили модель, чтобы предсказывать продажи машин по их параметрам и цене:  

\begin{split}
& \text{Sales}(f_1, ..., f_n, p) & \ : \ & \mathbb{R}^{n+1} \rightarrow \mathbb{R} \\
\end{split}
* $f_1, ..., f_n$ - параметры машины
* $p$ - цена

Задача формулируется так:  

\begin{split}
\textrm{maximize}  \quad  & p \cdot s  \\
\textrm{subject to}\quad  & s = \text{Sales}(f_1, ..., f_n, p) & \\
                          & f_i \cdot f_j <= \text{MinSize}    & i = \text{Length}, j = \text{Width}\\
                          & p,s \in \mathbb{R} & \\
                          & f_i \in \mathbb{R} & i \neq \text{VehicleType} \\
                          & f_i \in \{0, 1\} & i = \text{VehicleType} \\
\end{split}

In [1]:
from pyscipopt import Model
from pyscipopt_ml.add_predictor import add_predictor_constr
from sklearn.ensemble import RandomForestRegressor
import torch
import pandas as pd
import numpy as np

In [2]:
# Читаем датасэт
df = pd.read_csv('car_sales.csv')
df

,Manufacturer,Model,Sales_in_thousands,__year_resale_value,Vehicle_type,Price_in_thousands,Engine_size,Horsepower,Wheelbase,Width,Length,Curb_weight,Fuel_capacity,Fuel_efficiency,Latest_Launch,Power_perf_factor
0,Acura,Integra,16.919,16.360,Passenger,21.50,1.8,140,101.2,67.3,172.4,2.639,13.2,28,2/2/2012,58.280150
1,Acura,TL,39.384,19.875,Passenger,28.40,3.2,225,108.1,70.3,192.9,3.517,17.2,25,6/3/2011,91.370778
2,Acura,CL,14.114,18.225,Passenger,26.50,3.2,225,106.9,70.6,192.0,3.470,17.2,26,1/4/2012,91.300000
3,Acura,RL,8.588,29.725,Passenger,42.00,3.5,210,114.6,71.4,196.6,3.850,18.0,22,3/10/2011,91.389779
4,Audi,A4,20.397,22.255,Passenger,23.99,1.8,150,102.6,68.2,178.0,2.998,16.4,27,10/8/2011,62.777639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,Volvo,V40,3.545,17.200,Passenger,24.40,1.9,160,100.5,67.6,176.6,3.042,15.8,25,9/21/2011,66.498812
152,Volvo,S70,15.245,20.600,Passenger,27.50,2.4,168,104.9,69.3,185.9,3.208,17.9,25,11/24/2012,70.654495
153,Volvo,V70,17.531,20.800,Passenger,28.80,2.4,168,104.9,69.3,186.2,3.259,17.9,25,6/25/2011,71.155978
154,Volvo,C70,3.493,31.000,Passenger,45.50,2.3,236,104.9,71.5,185.7,3.601,18.5,23,4/26/2011,101.623357


In [3]:
# Будем использовать эти фичи
features = [
    'Vehicle_type',
    'Engine_size',
    'Horsepower',
    'Wheelbase',
    'Width',
    'Length',
    'Curb_weight',
    'Fuel_capacity',
    'Fuel_efficiency',
    'Power_perf_factor',
]

In [170]:
# Создаём модель прогнозирования продаж (torch)
sales = torch.tensor(df['Sales_in_thousands'], dtype=torch.float32)
df1 = df[features + ['Price_in_thousands']]
df1.loc[:, 'Vehicle_type'] = (df1['Vehicle_type'] == 'Passenger')
df1['Vehicle_type'] = df1['Vehicle_type'].astype("float")
# print(df1.dtypes)
X = torch.tensor(df1.to_numpy(), dtype=torch.float32)
# print(X)

n_inputs = X.shape[1]
layers_sizes = (20, 20, 10)
reg_sales = torch.nn.Sequential(
            torch.nn.Linear(n_inputs, layers_sizes[0]),
            torch.nn.Sigmoid(),
            torch.nn.Linear(layers_sizes[0], layers_sizes[1]),
            torch.nn.Sigmoid(),
            torch.nn.Linear(layers_sizes[1], layers_sizes[2]),
            torch.nn.ReLU(),
            torch.nn.Linear(layers_sizes[2], 1),
        )
# reg_sales(X)

C:\Users\maksim.goncharov\AppData\Local\Temp\ipykernel_16956\1917797997.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Vehicle_type'] = df1['Vehicle_type'].astype("float")


In [171]:
# Обучаем модель прогнозирования продаж (torch)
batch_size = X.shape[0]
n_epochs = 2000
n_train = X.shape[0]
n_batches = n_train // batch_size

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(reg_sales.parameters(), lr=0.001, weight_decay=0.0001)

epoch=0
for epoch in range(n_epochs):
    # print(f"epoch: {epoch}")
    t_loss = 0
    i = 0
    for batch_num in range(n_batches):
        # print(f"batch_num: {batch_num}")
        batch_X = X[batch_num * batch_size : (batch_num + 1) * batch_size, :]
        batch_sales = sales[batch_num * batch_size : (batch_num + 1) * batch_size]
        
        sales_pred = reg_sales(batch_X)
        loss = criterion(sales_pred, batch_sales)
        i += 1
        t_loss += loss.item()
    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(t_loss/i)

# print(criterion(sales, reg_sales(batch_X)).item())

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([156])) that is different to the input size (torch.Size([156, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


7429.04296875
7428.6572265625
7428.26611328125
7427.86572265625
7427.4541015625
7427.02880859375
7426.58642578125
7426.12646484375
7425.64892578125
7425.15966796875
7424.666015625
7424.1787109375
7423.69873046875
7423.22998046875
7422.76513671875
7422.30224609375
7421.8369140625
7421.3681640625
7420.8935546875
7420.4150390625
7419.93017578125
7419.43896484375
7418.9423828125
7418.4384765625
7417.92822265625
7417.41162109375
7416.8876953125
7416.3564453125
7415.818359375
7415.27294921875
7414.720703125
7414.16064453125
7413.59228515625
7413.01708984375
7412.4345703125
7411.84375
7411.244140625
7410.63623046875
7410.021484375
7409.39794921875
7408.76513671875
7408.1240234375
7407.474609375
7406.8154296875
7406.1484375
7405.47119140625
7404.7861328125
7404.09228515625
7403.3876953125
7402.673828125
7401.951171875
7401.21826171875
7400.4765625
7399.72314453125
7398.95947265625
7398.1845703125
7397.39892578125
7396.60205078125
7395.794921875
7394.97509765625
7394.1435546875
7393.29931640625

In [172]:
model = Model()
model.redirectOutput()

# Цена нашего продукта
lb_price = df['Price_in_thousands'].quantile(0.1)
ub_price = df['Price_in_thousands'].quantile(0.9)
x_price = model.addVar(vtype='C', lb=lb_price, ub=ub_price, name='price')

# Продажи нашего продукта
lb_sales = df['Sales_in_thousands'].quantile(0.1)
ub_sales = df['Sales_in_thousands'].quantile(0.9)
x_sales = model.addVar(vtype='C', lb=lb_sales, ub=ub_sales, name='sales')

# Выручка
# В SCIP нельзя задавать нелинейные целевые функции,
# поэтому используем трюк с введением доп. переменной
x_revenue = model.addVar(vtype='C', lb=lb_price * lb_sales, ub=ub_price * ub_sales, name='revenue')
model.setObjective(x_revenue, 'maximize')
model.addCons(x_revenue <= x_price * x_sales, name='revenue')

# Фичи нашего продукта
x_features = []
for feature in features:
    if feature == 'Vehicle_type':
        x_features.append(model.addVar(vtype='B', name=feature))
    else:
        lb = df[feature].quantile(0.1)
        ub = df[feature].quantile(0.9)
        x_features.append(model.addVar(vtype='C', lb=lb, ub=ub, name=feature))

# Допустим мы хотим, чтобы наши машины были не больше заданной площади
i_width = features.index('Width')
i_length = features.index('Length')
max_size = (df['Width'] * df['Length']).quantile(0.3)
model.addCons(x_features[i_width] * x_features[i_length] <= max_size, name='size')

# Ограничения, которые определяют переменную x_sales
add_predictor_constr(
    model,
    reg_sales,
    np.array(x_features + [x_price]).reshape((1, -1)),
    np.array([[x_sales]]),
    epsilon=0.0001,
    unique_naming_prefix='model_sales_',
)

In [173]:
print('=== Наши переменные ===')
for v in model.getVars():
    if 'model_' not in str(v):
        print(v, v.vtype())

=== Наши переменные ===
Vehicle_type BINARY
sales CONTINUOUS
revenue CONTINUOUS
price CONTINUOUS
Engine_size CONTINUOUS
Horsepower CONTINUOUS
Wheelbase CONTINUOUS
Width CONTINUOUS
Length CONTINUOUS
Curb_weight CONTINUOUS
Fuel_capacity CONTINUOUS
Fuel_efficiency CONTINUOUS
Power_perf_factor CONTINUOUS


In [174]:
print('=== Переменные, которые добавил фреймворк ===')
for v in model.getVars():
    if 'model_' in str(v):
        print(v, v.vtype())

=== Переменные, которые добавил фреймворк ===
model_sales_layer_0__0_0 CONTINUOUS
model_sales_layer_0__0_1 CONTINUOUS
model_sales_layer_0__0_2 CONTINUOUS
model_sales_layer_0__0_3 CONTINUOUS
model_sales_layer_0__0_4 CONTINUOUS
model_sales_layer_0__0_5 CONTINUOUS
model_sales_layer_0__0_6 CONTINUOUS
model_sales_layer_0__0_7 CONTINUOUS
model_sales_layer_0__0_8 CONTINUOUS
model_sales_layer_0__0_9 CONTINUOUS
model_sales_layer_0__0_10 CONTINUOUS
model_sales_layer_0__0_11 CONTINUOUS
model_sales_layer_0__0_12 CONTINUOUS
model_sales_layer_0__0_13 CONTINUOUS
model_sales_layer_0__0_14 CONTINUOUS
model_sales_layer_0__0_15 CONTINUOUS
model_sales_layer_0__0_16 CONTINUOUS
model_sales_layer_0__0_17 CONTINUOUS
model_sales_layer_0__0_18 CONTINUOUS
model_sales_layer_0__0_19 CONTINUOUS
model_sales_layer_1_0_0 CONTINUOUS
model_sales_layer_1_0_1 CONTINUOUS
model_sales_layer_1_0_2 CONTINUOUS
model_sales_layer_1_0_3 CONTINUOUS
model_sales_layer_1_0_4 CONTINUOUS
model_sales_layer_1_0_5 CONTINUOUS
model_sales_la

In [175]:
model.optimize()

presolving:
(round 1, fast)       14 del vars, 11 del conss, 0 add conss, 55 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       17 del vars, 11 del conss, 0 add conss, 142 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 3, fast)       17 del vars, 11 del conss, 0 add conss, 167 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 4, medium)     19 del vars, 19 del conss, 0 add conss, 176 chg bounds, 0 chg sides, 9 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 5, fast)       23 del vars, 21 del conss, 0 add conss, 176 chg bounds, 0 chg sides, 9 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 6, medium)     23 del vars, 23 del conss, 0 add conss, 176 chg bounds, 0 chg sides, 11 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
   (0.0s) probing cycle finished: starting next cycle
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetr

In [176]:
# Посмотрим на данные и на наш результат
df1 = pd.DataFrame(data={
    'revenue': df['Sales_in_thousands'] * df['Price_in_thousands'],
    'sales': df['Sales_in_thousands'],
    'price': df['Price_in_thousands'],
    'size': df['Width'] * df['Length'],
})
df1 = df1.sort_values(['revenue'], ascending=False)

entries = []
for i, feature in enumerate(features):
    if feature == 'Vehicle_type':
        label = ['Car', 'Passenger'][int(round(model.getVal(x_features[i])))]
        entries.append((feature, label, df[feature].min(), df[feature].max()))
    else:
        entries.append((feature, model.getVal(x_features[i]), df[feature].quantile(0.1), df[feature].quantile(0.9)))
our_features = pd.DataFrame(data=entries, columns=['feature', 'value', 'dataset_lb', 'dataset_ub'])
sales = model.getVal(x_sales)
price = model.getVal(x_price)
revenue = model.getVal(x_sales) * model.getVal(x_price)
size = model.getVal(x_features[features.index('Width')] * x_features[features.index('Length')])

print('=== Топ 10 продаж из датасэта ===')
print(df1.head(10))
print()
print('=== Ожидаемые продажи нашей машины ===')
print(f'revenue {revenue:.2f}, sales {sales:.2f}, price {price:.2f}, size {size:.2f}')
print()
print('=== Параметры нашей машины ===')
print(our_features)

=== Топ 10 продаж из датасэта ===
          revenue    sales   price      size
55   14560.010535  540.561  26.935  17757.95
51    8836.531710  276.747  31.930  13387.14
53    4529.088630  125.338  36.135  16102.02
39    4418.607060  227.061  19.460  17779.06
48    4396.401275  245.815  17.885  14424.80
136   4344.358892  247.994  17.518  13213.85
68    4223.590800  157.040  26.895  13122.45
44    3555.919185  181.749  19.565  14307.84
57    3544.345700  230.902  15.350  13272.64
52    3335.399670  155.787  21.410  15388.94

=== Ожидаемые продажи нашей машины ===
revenue 2326.61, sales 53.00, price 43.90, size 12543.39

=== Параметры нашей машины ===
             feature       value dataset_lb  dataset_ub
0       Vehicle_type   Passenger        Car   Passenger
1        Engine_size        1.95       1.95         4.6
2         Horsepower       254.0      120.0       254.0
3          Wheelbase       115.5      98.65       115.5
4              Width        66.9       66.9        76.2
5     